In [2]:
import rasterio
from rasterio.plot import show
from rasterio.enums import Resampling
import pyproj
from matplotlib import pyplot
import numpy as np
import rioxarray
from rioxarray import open_rasterio
import os
from pathlib import Path

In [3]:
def resample_raster(src_raster, target_raster_profile, resampling_method=Resampling.nearest):
    """
    Resample the source raster to match the target raster's profile.
    """
    with rasterio.open(src_raster) as src:
        data = np.empty((src.count, target_raster_profile['height'], target_raster_profile['width']), dtype=src.dtypes[0])
        transform = target_raster_profile['transform']
        
        for i in range(1, src.count + 1):
            rasterio.warp.reproject(
                source=rasterio.band(src, i),
                destination=data[i - 1],
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=target_raster_profile['crs'],
                resampling=resampling_method
            )
        return data[0], transform



In [4]:
def reproject_to_wgs84(file_path,target_path):
    with open_rasterio(file_path) as img:
        reprojected_img = img.rio.reproject('EPSG:4326')
        reprojected_img.rio.to_raster(target_path)


In [42]:
for filename in os.listdir('./data'):
    if 'cropscape-strawberries' in filename:
        file_path=os.path.join('./data', filename)
        if os.path.isfile(file_path):
            target_path=os.path.join('./data', f'converted-{filename}')
            reproject_to_wgs84(file_path, target_path)
            print(f'Processed {file_path} to {target_path}')

Processed ./data/cropscape-strawberries-06053-2016.tif to ./data/converted-cropscape-strawberries-06053-2016.tif
Processed ./data/cropscape-strawberries-06053-2017.tif to ./data/converted-cropscape-strawberries-06053-2017.tif
Processed ./data/cropscape-strawberries-06053-2015.tif to ./data/converted-cropscape-strawberries-06053-2015.tif
Processed ./data/cropscape-strawberries-06053-2014.tif to ./data/converted-cropscape-strawberries-06053-2014.tif
Processed ./data/cropscape-strawberries-06053-2010.tif to ./data/converted-cropscape-strawberries-06053-2010.tif
Processed ./data/cropscape-strawberries-06053-2011.tif to ./data/converted-cropscape-strawberries-06053-2011.tif
Processed ./data/cropscape-strawberries-06053-2013.tif to ./data/converted-cropscape-strawberries-06053-2013.tif
Processed ./data/cropscape-strawberries-06053-2012.tif to ./data/converted-cropscape-strawberries-06053-2012.tif
Processed ./data/cropscape-strawberries-06053-2023.tif to ./data/converted-cropscape-strawberrie

In [8]:
for filename in os.listdir('./data/landsat_monterey_extracted'):
    if '.tif' in filename:
        file_path=os.path.join('./data/landsat_monterey_extracted', filename)
        converted_dir = Path('./data/landsat_monterey_extracted/converted')
        converted_dir.mkdir(exist_ok=True)
        if os.path.isfile(file_path):
            target_path=os.path.join('./data/landsat_monterey_extracted/converted', f'converted-{filename}')
            reproject_to_wgs84(file_path, target_path)
            print(f'Processed {file_path} to {target_path}')
print('completed')

Processed ./data/landsat_monterey_extracted/LC08_L2SP_043035_20140223_20200911_02_T1_SR_B1.tif to ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20140223_20200911_02_T1_SR_B1.tif
Processed ./data/landsat_monterey_extracted/LC08_L2SP_043035_20221128_20221206_02_T1_QA_PIXEL.tif to ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20221128_20221206_02_T1_QA_PIXEL.tif
Processed ./data/landsat_monterey_extracted/LC08_L2SP_043035_20180509_20200901_02_T1_ETF.tif to ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20180509_20200901_02_T1_ETF.tif
Processed ./data/landsat_monterey_extracted/LC09_L2SP_043035_20220512_20230417_02_T1_ETUN.tif to ./data/landsat_monterey_extracted/converted/converted-LC09_L2SP_043035_20220512_20230417_02_T1_ETUN.tif
Processed ./data/landsat_monterey_extracted/LC08_L2SP_043035_20240219_20240229_02_T1_ST_EMSD.tif to ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20240219_20240229

In [9]:

#define function to perform masking
def mask_tif(target_path, source_path, output_path):
    with rasterio.open(target_path) as target:
        target_array = target.read()
        target_profile = target.profile

    #resample source to match the target
    source_resampled, _ = resample_raster(source_path, target_profile)

    #set masking
    threshold = 221
    mask_array = source_resampled == threshold

    # Apply the mask
    masked_array = np.where(mask_array, target_array, 0)

    # Save the masked array to a new TIFF file
    with rasterio.open(output_path, 'w', **target_profile) as dst:
        for i in range(target_array.shape[0]):
            dst.write(masked_array[i].astype('float32'), i+1)

    print(f"Masked TIFF file saved at {output_path}")



In [10]:
#target directory is the landsat evi files directory, source directory is the masked cropscape directory
target_dir = "./data/landsat_monterey_extracted/converted"
source_dir = "./data"
dest_dir = "./data/landsat_monterey_masked"

# Check if destination directory exists, create if it doesn't
if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

# Iterate over each .tif file in the target directory
for target_filename in os.listdir(target_dir):
    if target_filename.endswith('.tif'):
        target_file = os.path.join(target_dir, target_filename)
        year=target_filename[27:31]
        print("Landsat Year:", year)
        source_file = os.path.join(source_dir, f"converted-cropscape-strawberries-06053-{year}.tif")
        output_file = os.path.join(dest_dir, f"{os.path.splitext(target_filename)[0]}_masked.tiff")

        # Check if source file exists
        if os.path.exists(source_file):
            print(f"Processing {target_file} and {source_file}")
            mask_tif(target_file, source_file, output_file)
        else:
            print(f"Source file not found for {target_file}: {source_file}")

print("Processing complete.")

Landsat Year: 2015
Processing ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20150226_20200910_02_T1_ETF.tif and ./data/converted-cropscape-strawberries-06053-2015.tif
Masked TIFF file saved at ./data/landsat_monterey_masked/converted-LC08_L2SP_043035_20150226_20200910_02_T1_ETF_masked.tiff
Landsat Year: 2021
Processing ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20210602_20210608_02_T1_SR_B6.tif and ./data/converted-cropscape-strawberries-06053-2021.tif
Masked TIFF file saved at ./data/landsat_monterey_masked/converted-LC08_L2SP_043035_20210602_20210608_02_T1_SR_B6_masked.tiff
Landsat Year: 2024
Source file not found for ./data/landsat_monterey_extracted/converted/converted-LC09_L2SP_043035_20240314_20240315_02_T1_SR_B7.tif: ./data/converted-cropscape-strawberries-06053-2024.tif
Landsat Year: 2022
Processing ./data/landsat_monterey_extracted/converted/converted-LC08_L2SP_043035_20220621_20220705_02_T1_SR_NDMI.tif and ./data/converted-